In [1]:
import torch
import torch.nn as nn
import torch.optim as optim # 최적화 알고리즘
import torch.nn.init as init # 가중치 초기화
import torch.nn.functional as F # 함수 가져올때
import torchvision.transforms as transforms # 이미지 전처리할때
import torchvision.datasets as dset

from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt

In [2]:
batch_size = 256
learning_rate = 0.0002 # 너무 낮게하면 학습속도가 너무 낮아짐. 조절 필요!!
num_epoch = 10

In [3]:
mnist_train = dset.MNIST(root="../", train = True, transform=transforms.ToTensor(),
                         target_transform = None, download = True)
mnist_test = dset.MNIST(root="../", train =False, transform=transforms.ToTensor(),
                        target_transform = None, download = True)

100%|██████████| 9912422/9912422 [00:00<00:00, 123348833.86it/s]


Extracting ../MNIST/raw/train-images-idx3-ubyte.gz to ../MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 48983297.14it/s]


Extracting ../MNIST/raw/train-labels-idx1-ubyte.gz to ../MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 41810094.77it/s]

Extracting ../MNIST/raw/t10k-images-idx3-ubyte.gz to ../MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 18057373.24it/s]


Extracting ../MNIST/raw/t10k-labels-idx1-ubyte.gz to ../MNIST/raw



In [4]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
print(mnist_test.__getitem__(0)[0].size(), mnist_test.__len__())

torch.Size([1, 28, 28]) 60000
torch.Size([1, 28, 28]) 10000


In [5]:
train_loader = DataLoader(mnist_train,batch_size=batch_size, shuffle=True,
                          num_workers=2,drop_last=True)

test_loader = DataLoader(mnist_test,batch_size=batch_size, shuffle=False,
                         num_workers=2,drop_last=True)

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()


        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5), #16 *24 *24
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) #16 *12 *12
        )


        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5),#32 *8 *8
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) #32 *4 *4
        )


        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2), #64 *4 *4
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) # 64 *2 *2
        )

        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )


        self.fc1 = nn.Linear(256, 100)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(100, 10)


    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


model = CNN().to(device)

loss_func = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

cpu


In [9]:
loss_arr =[]
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)

        optimizer.zero_grad() #초기화
        output = model.forward(x)
        loss = loss_func(output,y_)
        loss.backward() #역전파
        optimizer.step()

        if j % 1000 == 0:
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())

tensor(2.3086, grad_fn=<NllLossBackward0>)
tensor(0.5288, grad_fn=<NllLossBackward0>)
tensor(0.2239, grad_fn=<NllLossBackward0>)
tensor(0.2300, grad_fn=<NllLossBackward0>)
tensor(0.1498, grad_fn=<NllLossBackward0>)
tensor(0.1631, grad_fn=<NllLossBackward0>)
tensor(0.1191, grad_fn=<NllLossBackward0>)
tensor(0.1308, grad_fn=<NllLossBackward0>)
tensor(0.1348, grad_fn=<NllLossBackward0>)
tensor(0.1161, grad_fn=<NllLossBackward0>)


In [10]:
correct = 0
total = 0


with torch.no_grad():

    for image,label in test_loader:


        x = image.to(device)
        y_= label.to(device)

        output = model.forward(x)

        _,output_index = torch.max(output,1)

        total += label.size(0)

        correct += (output_index == y_).sum().float()

    print("Accuracy of Test Data: {}%".format(100*correct/total))

Accuracy of Test Data: 97.73638153076172%
